In [2]:
import sys,os
current_dir = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(current_dir)))
from utils.Audio import PrintAudioInfo, GetAudio, remove_Silence
from utils.levenshtein_distance import infer
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor
import torch
import numpy as np


model_id = 'hongseongpil/wav2vec2-vocals'
model = Wav2Vec2ForCTC.from_pretrained(model_id,output_attentions=True)
model.eval()
processor = Wav2Vec2Processor.from_pretrained(model_id)

from utils.Tokenize_Kor import decompose_tokens
audio = remove_Silence(GetAudio("곰세마리.wav")[:1000*20])
input = processor(np.array(audio.set_channels(1).get_array_of_samples(),dtype=np.float32), sampling_rate=16000, return_tensors="pt").input_values[0]
with torch.no_grad():
    input_values = torch.tensor(input).unsqueeze(0)
    logits = model(input_values).logits

predlogits = torch.argmax(logits, dim=-1)[0]
outputs = processor.decode(predlogits,output_char_offsets=True)

origintext = "곰 세마리가 한집에 있어 아빠곰 엄마곰 애기곰 아빠곰은 뚱뚱해 엄마 곰"
origintext = "".join(decompose_tokens(origintext)).replace(" ","")
predtext = outputs['text'].replace(" ","")

predlogits = torch.argmax(logits, dim=-1)[0]
outputs = processor.decode(predlogits,output_char_offsets=True)
time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate

Some weights of the model checkpoint at hongseongpil/wav2vec2-vocals were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at hongseongpil/wav2vec2-vocals and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this mode

In [3]:
infered = infer(origintext,predtext)

In [37]:
print("origintext:", origintext)
print("predtext:",predtext)

print("Matching....\n")

for item in infered:
    pred ="".join([predtext[j-1] for j in item[1]])
    print("origintext:", origintext[item[0]-1],"predtext:",pred)

for w in outputs['char_offsets']:
    start_offset = round (w[ "start_offset" ] * time_offset, 2 )
    end_offset = round (w[ "end_offset" ] * time_offset, 2 )
    print('word : ',w['char'],'start_offset :',start_offset,'end_offset :',end_offset)
    # display(audio[start_offset*1000:end_offset*1000])


origintext: ㄱㅗㅁㅅㅔㅁㅏㄹㅣㄱㅏㅎㅏㄴㅈㅣㅂㅔㅇㅣㅆㅓㅇㅏㅃㅏㄱㅗㅁㅇㅓㅁㅁㅏㄱㅗㅁㅇㅐㄱㅣㄱㅗㅁㅇㅏㅃㅏㄱㅗㅁㅡㄴㄸㅜㅇㄸㅜㅇㅎㅐㅇㅓㅁㅁㅏㄱㅗㅁ
predtext: ㄱㅓㅇㅓㄴㅅㅔㅇㅁㅏㅇㅏㄹㅣㅇㅣㄱㅏㅇㅏㅎㅏㅇㅏㅇㅈㅣㅂㅔㅇㅣㅆㅓㅇㅏㅇㅏㅃㅏㄱㅗㅇㅗㅁㅇㅓㅁㅏㄱㅓㅇㅗㅁㅇㅐㅂㅎㄹㅇㅣㄱㅗㅇㅗㅇㅏㅃㅏㅇㅏㄱㅗㅇㅗㅁㅡㅇㅡㄴㄸㅜㅇㅜㅁㅌㅜㅇㅜㅁㅎㅐㅇㅐㅇㅓㅁㅁㅏㅇㅏㄱㅓㅇㅓㅓ
Matching....

origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅓㅇㅓ
origintext: ㅁ predtext: ㄴ
origintext: ㅅ predtext: ㅅ
origintext: ㅔ predtext: ㅔㅇ
origintext: ㅁ predtext: ㅁ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄹ predtext: ㄹ
origintext: ㅣ predtext: ㅣㅇㅣ
origintext: ㄱ predtext: ㄱ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㅎ predtext: ㅎ
origintext: ㅏ predtext: ㅏㅇㅏ
origintext: ㄴ predtext: ㅇ
origintext: ㅈ predtext: ㅈ
origintext: ㅣ predtext: ㅣ
origintext: ㅂ predtext: ㅂ
origintext: ㅔ predtext: ㅔ
origintext: ㅇ predtext: ㅇ
origintext: ㅣ predtext: ㅣ
origintext: ㅆ predtext: ㅆ
origintext: ㅓ predtext: ㅓ
origintext: ㅇ predtext: ㅇㅏㅇ
origintext: ㅏ predtext: ㅏ
origintext: ㅃ predtext: ㅃ
origintext: ㅏ predtext: ㅏ
origintext: ㄱ predtext: ㄱ
origintext: ㅗ predtext: ㅗㅇㅗ
origintext: ㅁ predtext: ㅁ
origintext: ㅇ predt